In [ ]:
pip install sentence_transformers

     |████████████████████████████████| 78 kB 6.0 MB/s 
     |████████████████████████████████| 3.4 MB 34.5 MB/s 
     |████████████████████████████████| 6.8 MB 52.8 MB/s 
     |████████████████████████████████| 1.2 MB 60.1 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 895 kB 54.4 MB/s 
     |████████████████████████████████| 3.3 MB 52.5 MB/s 
     |████████████████████████████████| 596 kB 59.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=d21a43a7e68c0fa5bc38847c330660e024f457a83d3a4575629afb48f41a6711
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

gpu num:  1


In [ ]:
embedder = SentenceTransformer('bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import json
from tqdm import tqdm_notebook as tqdm
import pickle

In [ ]:
import math 
mylines = []                             # Declare an empty list named mylines.
with open ('script.txt', 'rt') as myfile: # Open lorem.txt for reading text data.
    for myline in myfile:                # For each line, stored as myline,
        mylines.append(myline)           # add its contents to mylines.
mydialogue = []
for i in range(len(mylines)):
    if len(mylines[i])- len(mylines[i].lstrip())>9:
        mydialogue.append(mylines[i])

mydial = ''.join(mydialogue)                    # In this part what i do is that I put in the good form for the future preprocessing operations
dial = mydial.split('                     ')    # I take out all the blank spaces and \n and I do a list of list and each list is a sentence said by a new character
dialogue = [s.replace("\n", "") for s in dial]
dialoguetot = []
for i in range (len(dialogue)):
    dialogue_i = []
    dialogue_i.append(dialogue[i])
    dialoguetot.append(dialogue_i)

In [ ]:
realdialogue = []  #made to take out of dialogue all empty strings
for i in range (len(dialoguetot)):
  if any(c.isalpha() for c in dialoguetot[i][0]) == True:
    realdialogue.append(dialoguetot[i])
  else : 
    pass



In [ ]:
realdialogue

[['    GLADIATOR'],
 ['    by                    David Franzoni               Revised by John Logan'],
 ['                   SECOND DRAFT'],
 ['                   October 22, 1998          "While stands the Colosseum, Rome shall stand.          When falls the Colosseum, Rome shall fall.          And when Rome falls -- the World."'],
 ['              Byron'],
 ["    COMMODUS          Do you think he's really dying?"],
 ["    LUCILLA          He's been dying for ten years."],
 ["    COMMODUS          I think he's really dying this time."],
 ['    COMMODUS          He has to be bled every night now.'],
 ['    LUCILLA          How do you know that?'],
 ["    COMMODUS          I've been so informed."],
 ["    COMMODUS          If he weren't really dying he          wouldn't have sent for us."],
 ['    LUCILLA               (a smile)          Maybe he just misses us.'],
 ["    COMMODUS          And the Senators.  He wouldn't have          summoned them if --"],
 ['    LUCILLA          Peace,

In [ ]:
# this is to group our sentences into group of 10 sentences
conv = [realdialogue[0]]
conv1 = []
for i in range (1, len(realdialogue)):
  if i % 10 == 0:
    conv1.append(realdialogue[i])
    conv.append(conv1)
    conv1 = []
  else :
    conv1.append(realdialogue[i])


In [ ]:
# Test of a new way to divide the movie into three parts as beginning middle end 
import math
n = len(realdialogue)
conv1 = []
conv2 = []
conv3 = []
for i in range(0,math.ceil(n/3)):
  conv1.append(realdialogue[i])
for i in range(math.ceil(n/3), math.ceil((2*n)/3)):
  conv2.append(realdialogue[i])
for i in range(math.ceil((2*n)/3),n):
  conv3.append(realdialogue[i])
conv = [conv1,conv2,conv3]

In [ ]:
conv

[[['    GLADIATOR'],
  ['    by                    David Franzoni               Revised by John Logan'],
  ['                   SECOND DRAFT'],
  ['                   October 22, 1998          "While stands the Colosseum, Rome shall stand.          When falls the Colosseum, Rome shall fall.          And when Rome falls -- the World."'],
  ['              Byron'],
  ["    COMMODUS          Do you think he's really dying?"],
  ["    LUCILLA          He's been dying for ten years."],
  ["    COMMODUS          I think he's really dying this time."],
  ['    COMMODUS          He has to be bled every night now.'],
  ['    LUCILLA          How do you know that?'],
  ["    COMMODUS          I've been so informed."],
  ["    COMMODUS          If he weren't really dying he          wouldn't have sent for us."],
  ['    LUCILLA               (a smile)          Maybe he just misses us.'],
  ["    COMMODUS          And the Senators.  He wouldn't have          summoned them if --"],
  ['    LUCILLA 

In [ ]:
conv[5]

[['  DMITRI          Go!'],
 ['  DMITRI (CONT.)          Go!'],
 ["  DMITRI (CONT.)                  (lying to the mob)          No one's here! Let's try the          next room!"],
 ['  ANASTASIA          Hurry, Grandmama!'],
 ['  TATIANA          Get on! Anastasia, get on!'],
 ['  TATIANA          Grab my hand!'],
 ["  ANASTASIA          Don't let go!"],
 ['  TATIANA (O.C.)          ANASTASIA!'],
 ['  TATIANA          ANASTASIA!'],
 ['   SUPERIMPOSE:']]

In [ ]:
# this is to make list of lists because before it was a list of lists of lists 
realconv =[]
for i in range(0, len(conv)):
  flat_conv = []
  for sublist in conv[i]:
    for item in sublist:
        flat_conv.append(item)
  realconv.append(flat_conv)

In [ ]:
realconv

[['    GLADIATOR',
  '    by                    David Franzoni               Revised by John Logan',
  '                   SECOND DRAFT',
  '                   October 22, 1998          "While stands the Colosseum, Rome shall stand.          When falls the Colosseum, Rome shall fall.          And when Rome falls -- the World."',
  '              Byron',
  "    COMMODUS          Do you think he's really dying?",
  "    LUCILLA          He's been dying for ten years.",
  "    COMMODUS          I think he's really dying this time.",
  '    COMMODUS          He has to be bled every night now.',
  '    LUCILLA          How do you know that?',
  "    COMMODUS          I've been so informed.",
  "    COMMODUS          If he weren't really dying he          wouldn't have sent for us.",
  '    LUCILLA               (a smile)          Maybe he just misses us.',
  "    COMMODUS          And the Senators.  He wouldn't have          summoned them if --",
  '    LUCILLA          Peace, Commodus.  Af

In [ ]:
def encode_conversation(data, profix):
        
    
    embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(data))):
            #tokens_input = tokenize_conv(sent[i]).cuda()
            embedding = embedder.encode(data[i])

            embeddings.append(embedding)
            
    with open(profix + '_sentence_transformer.pkl', 'wb') as f:
        pickle.dump(embeddings, f)
    
        
    return embeddings

In [ ]:
embeddings = encode_conversation(realconv,'gladiator')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
embeddings = encode_conversation('val')

/nethome/jchen896/anaconda3/envs/new_torch/lib/python3.6/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


In [ ]:
embeddings = encode_conversation('test')

/nethome/jchen896/anaconda3/envs/new_torch/lib/python3.6/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()
